# Blood Pressure Trends Over Time

This notebook generates a chart of blood pressure readings over time from blood pressure data held in the Health Tracker database and retrieved via the Health Tracker Web Service.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, datetime, timedelta

# Reporting date range
start = date(1900, 1, 1)
end = datetime.now() + timedelta(days=1)

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_daily_blood_pressure_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
# The agg() function will fail if there are non-numeric columns in the data frame, aside from
# the date. Extract the data for aggregation
numeric_df = df[["date", "systolic", "diastolic"]]

# Make sure the date is the index
numeric_df.set_index("date", inplace=True)

# Resample the data, weekly and monthly, including all periods (even if there are gaps). This results
# in gaps in the plot, that accurately reflect the data
monthly = numeric_df.resample("ME").agg(["mean", "min", "max"]).asfreq("ME")

In [ ]:
# Export the data to a spreadsheet
if export_spreadsheet:
    export_to_spreadsheet("blood_pressure", "blood_pressure_trends", { "Monthly Trends": monthly })


In [ ]:
import matplotlib.pyplot as plt

# Plot the systolic trends
fig, axs = plt.subplots(figsize=(12, 8))

monthly["systolic"]["mean"].plot(linewidth=2, label="Systolic Mean", color="orange")
monthly["systolic"]["min"].plot(linestyle="--", linewidth=2, label="Systolic Min", color="red")
monthly["systolic"]["max"].plot(linewidth=2, label="Systolic Max", color="red")

# Set title and labels
plt.title("Systolic Blood Pressure Trends Over Time")
plt.ylabel("Blood Pressure (mmHg)")
plt.grid(True)

# Move legend below the chart
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("blood_pressure", "blood_pressure_trends", "Monthly-Systolic", chart_export_format)

# Show the chart
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot diastolic trends
fig, axs = plt.subplots(figsize=(12, 8))

monthly["diastolic"]["mean"].plot(linewidth=2, label="Diastolic Mean", color="cyan")
monthly["diastolic"]["min"].plot(linestyle="--", linewidth=2, label="Diastolic Min", color="blue")
monthly["diastolic"]["max"].plot(linewidth=2, label="Diastolic Max", color="blue")

# Set title and labels
plt.title("Diastolic Blood Pressure Trends Over Time")
plt.ylabel("Blood Pressure (mmHg)")
plt.grid(True)

# Move legend below the chart
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=2)
plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("blood_pressure", "blood_pressure_trends", "Monthly-Diastolic", chart_export_format)

# Show the chart
plt.show()
